                                                                                                         Saurabh Bajoria
                                                                                                         UBid: sbajoria
                                                                                                         Person no:50208005

**CSE 4/587 Data Intensive Computing**
====================
***LAB2: DATA CLEANING AND MUNGING***
---------------------------------------
                                                                                                       

*Activity 2: Extract and repurpose data*
--------------------------------------

*Kagge Sqlite European Soccer Database*


- Note:
 1. Used Sqlite to make connection to the database
 2. Used read.csv.sql() function to read the data from CSV
 3. Used write.csv() to write data to a csv
 4. Used dbGetQuery to fetch the data from database tables
 5. Created two CSVs: 1. Player.csv   2. Team.csv
 

- Installing and Importing Libraries required

In [1]:
install.packages("dplyr", repos = "http://cran.us.r-project.org")
install.packages("sqldf", repos = "http://cran.us.r-project.org")

Installing package into 'C:/Users/Saurabh/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'dplyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Saurabh\AppData\Local\Temp\RtmpWed7EX\downloaded_packages


Installing package into 'C:/Users/Saurabh/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'sqldf' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Saurabh\AppData\Local\Temp\RtmpWed7EX\downloaded_packages


In [2]:
library(dplyr)
library(sqldf)

Warning message:
"package 'dplyr' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'sqldf' was built under R version 3.3.3"Loading required package: gsubfn
Loading required package: proto
Could not load tcltk.  Will use slower R code instead.
Loading required package: RSQLite


- Establishing a connection to the database using SQLite

In [3]:
db <- dbConnect(SQLite(), dbname="database.sqlite")

- Quering the database for creating the Player.csv
 - Joining the tables Player and Player_Attributes on player_api_id
 - Calculating Max overall rating for every player using group by player_api_id and player_name
 - Calculating Max sum of GK attributes for every player using group by player_api_id and player_name
 - Joining the tables Player and Match on player_api_id and Home_player1, Home_player2....Away_Player1, Away_Player2...
 - Using the above joined tables (Player and Match) to form a subquery
 - Count the number of appearances and distinct leagues using group by player_api_id and player_name

In [4]:
write.csv((dbGetQuery(conn = db,"select pa.player_api_id as 'player_id',REPLACE(p.player_name,',',';') as 'player_name',
max(pa.overall_rating) as 'overall_rating',
max(gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes) as 'gk_attributes',
Count_appearances.appearances as 'number_of_appearances',
Count_appearances.leagues as 'number_of_leagues'
from Player_Attributes pa, Player p, (select p.player_api_id,count(p.player_api_id) as 'appearances', count(distinct league_id) as 'leagues' 
from Player p, Match m
where p.player_api_id=m.home_player_1 OR
p.player_api_id=m.home_player_2 OR
p.player_api_id=m.home_player_3 OR
p.player_api_id=m.home_player_4 OR
p.player_api_id=m.home_player_5 OR
p.player_api_id=m.home_player_6 OR
p.player_api_id=m.home_player_7 OR
p.player_api_id=m.home_player_8 OR
p.player_api_id=m.home_player_9 OR
p.player_api_id=m.home_player_10 OR
p.player_api_id=m.home_player_11 OR
p.player_api_id=m.away_player_1 OR
p.player_api_id=m.away_player_2 OR
p.player_api_id=m.away_player_3 OR
p.player_api_id=m.away_player_4 OR
p.player_api_id=m.away_player_5 OR
p.player_api_id=m.away_player_6 OR
p.player_api_id=m.away_player_7 OR
p.player_api_id=m.away_player_8 OR
p.player_api_id=m.away_player_9 OR
p.player_api_id=m.away_player_10 OR
p.player_api_id=m.away_player_11
group by p.player_api_id) as 'Count_appearances'
where pa.player_api_id=p.player_api_id
and pa.player_api_id=Count_appearances.player_api_id
group by pa.player_api_id")),file = "Player.csv",row.names=FALSE)

- **Question 1:** Which are the top 10 players by overall rating?

In [5]:
read.csv.sql("Player.csv", 
    sql = "select player_id,player_name,overall_rating from file order by overall_rating desc LIMIT 10",
eol="\n")

Warning message:
"closing unused connection 5 (Player.csv)"Warning message:
"closing unused connection 4 (Player.csv)"

player_id,player_name,overall_rating
30981,"""Lionel Messi""",94
30717,"""Gianluigi Buffon""",93
30829,"""Wayne Rooney""",93
30893,"""Cristiano Ronaldo""",93
39854,"""Xavi Hernandez""",92
39989,"""Gregory Coupet""",92
30626,"""Thierry Henry""",91
30627,"""John Terry""",91
30657,"""Iker Casillas""",91
30723,"""Alessandro Nesta""",91


- **Question 2:** Which are the top 10 goalkeepers by sum of gk attributes?

In [6]:
read.csv.sql("Player.csv", 
    sql = "select player_id,player_name,gk_attributes from file order by gk_attributes desc LIMIT 10",
eol="\n")

Warning message:
"closing unused connection 4 (Player.csv)"

player_id,player_name,gk_attributes
30717,"""Gianluigi Buffon""",449
39989,"""Gregory Coupet""",447
30859,"""Petr Cech""",445
30657,"""Iker Casillas""",442
27299,"""Manuel Neuer""",440
30989,"""Julio Cesar""",438
24503,"""Sebastian Frey""",437
30726,"""Edwin van der Sar""",436
182917,"""David De Gea""",429
30660,"""Pepe Reina""",428


- **Question 3:** Which are the top 10 players by number of appearances (with any team they played with)?

In [7]:
read.csv.sql("Player.csv", 
    sql = "select player_id,player_name,number_of_appearances from file order by number_of_appearances desc LIMIT 10",
eol="\n")

Warning message:
"closing unused connection 6 (Player.csv)"Warning message:
"closing unused connection 4 (Player.csv)"

player_id,player_name,number_of_appearances
31293,"""Steve Mandanda""",300
41097,"""Stephane Ruffier""",294
33764,"""Gorka Iraizoz Moreno""",286
42422,"""Samir Handanovic""",286
26295,"""Hugo Lloris""",282
31465,"""Tim Howard""",282
31432,"""Joe Hart""",275
26168,"""Cedric Carrasso""",268
26399,"""Daniel Congre""",262
94550,"""Moussa Sissoko""",260


- **Question 4:** Which are the top 10 players by number of leagues they played in?

In [8]:
read.csv.sql("Player.csv", 
    sql = "select player_id,player_name,number_of_leagues from file order by number_of_leagues desc LIMIT 10",
eol="\n")

Warning message:
"closing unused connection 4 (Player.csv)"

player_id,player_name,number_of_leagues
42119,"""Gelson Fernandes""",6
116750,"""Ricky van Wolfswinkel""",5
178538,"""Filip Djuricic""",5
22543,"""Radamel Falcao""",4
24123,"""Jonathan De Guzman""",4
25816,"""Almen Abdi""",4
26434,"""Ibrahim Afellay""",4
26489,"""Edson Braafheid""",4
26675,"""Eljero Elia""",4
30477,"""Danijel Pranjic""",4


- Quering the database for creating the Team.csv
 - Joining the tables Team and Team_Attributes on player_api_id
 - Calculating Max BuildUp points for every player using group by player_api_id and player_name
 - Calculating Max ChanceCreating points for every player using group by player_api_id and player_name
 - Calculating Max Defense points for every player using group by player_api_id and player_name
 - Replacing NULL values for the above fields with 0
 - Selecting Home_team_id and home_team_goals and performing a union all with away_team_id and away_team_goals
 - Grouping by team_id and counting he goals for each team

In [9]:
write.csv((dbGetQuery(conn = db,"select teamName_goals.team_api_id,REPLACE(teamName_goals.team_long_name,',',';') as 'team_name',
teamName_goals.goals,
max(COALESCE(ta.buildUpPlaySpeed,0)+COALESCE(ta.buildUpPlayDribbling, 0)+COALESCE(ta.buildUpPlayPassing,0)) as 'build_up',
max(COALESCE(ta.chanceCreationPassing,0)+COALESCE(ta.chanceCreationCrossing,0)+COALESCE(ta.chanceCreationShooting,0)) as 'chance_creation',
max(COALESCE(ta.defencePressure,0)+COALESCE(ta.defenceAggression,0)+COALESCE(defenceTeamWidth,0)) as 'defense'
from (select team_goals.team_api_id,t.team_long_name,sum(team_goals.team_goals) as 'goals'
    from (select home_team_api_id as 'team_api_id', home_team_goal as 'team_goals'from Match m1
    union all 
    select away_team_api_id as 'team_api_id', away_team_goal as 'team_goals' from Match m2) as 'team_goals' inner join Team t
    on team_goals.team_api_id=t.team_api_id
    group by team_goals.team_api_id,t.team_long_name) as 'teamName_goals'
left join Team_attributes ta on ta.team_api_id=teamName_goals.team_api_id
group by teamName_goals.team_api_id,teamName_goals.team_long_name")),
file = "Team.csv",row.names=FALSE)

- **Question 5:** Which are the top 10 teams by sum of build up play attributes?

In [10]:
read.csv.sql("Team.csv", 
    sql = "select team_api_id,team_name ,build_up as 'build_up_attributes' from file order by build_up desc LIMIT 10",
eol="\n")

team_api_id,team_name,build_up_attributes
9804,"""Torino""",209
8600,"""Udinese""",205
8194,"""Fortuna D<fc>sseldorf""",198
8033,"""Podbeskidzie Bielsko-Biala""",193
8245,"""Korona Kielce""",192
8024,"""Widzew L<f3>dz""",190
8244,"""Widzew L<f3>dz""",190
208931,"""Carpi""",190
9830,"""FC Nantes""",189
7943,"""Sassuolo""",188


- **Question6:** Which are the top 10 teams by sum of chance creation attributes?

In [11]:
read.csv.sql("Team.csv", 
    sql = "select team_api_id,team_name ,chance_creation as 'chance_creation_attributes' from file order by chance_creation desc LIMIT 10",
eol="\n")

Warning message:
"closing unused connection 7 (Team.csv)"Warning message:
"closing unused connection 6 (Team.csv)"Warning message:
"closing unused connection 5 (Team.csv)"Warning message:
"closing unused connection 4 (Team.csv)"

team_api_id,team_name,chance_creation_attributes
8543,"""Lazio""",220
8697,"""SV Werder Bremen""",216
9875,"""Napoli""",215
8178,"""Bayer 04 Leverkusen""",210
8191,"""Burnley""",210
8344,"""Cardiff City""",210
8456,"""Manchester City""",210
8485,"""Aberdeen""",210
8548,"""Rangers""",210
8586,"""Tottenham Hotspur""",210


- **Question 7:** Which are the top 10 teams by sum of defense attributes?

In [12]:
read.csv.sql("Team.csv", 
    sql = "select team_api_id,team_name ,defense as 'defense_attributes' from file order by defense desc LIMIT 10",
eol="\n")

team_api_id,team_name,defense_attributes
1957,"""Jagiellonia Bialystok""",210
7794,"""SC Bastia""",210
8021,"""Zaglebie Lubin""",210
8197,"""Leicester City""",210
8344,"""Cardiff City""",210
8350,"""1. FC Kaiserslautern""",210
8466,"""Southampton""",210
8481,"""AS Nancy-Lorraine""",210
8483,"""Blackpool""",210
8521,"""Stade Brestois 29""",210


- **Question 8:** Which are the top 10 teams by number of scored goals?

In [13]:
read.csv.sql("Team.csv", 
    sql = "select team_api_id,team_name,goals from file order by goals desc LIMIT 10",
eol="\n")

Warning message:
"closing unused connection 6 (Team.csv)"Warning message:
"closing unused connection 5 (Team.csv)"Warning message:
"closing unused connection 4 (Team.csv)"

team_api_id,team_name,goals
8634,"""FC Barcelona""",849
8633,"""Real Madrid CF""",843
9925,"""Celtic""",695
9823,"""FC Bayern Munich""",653
8640,"""PSV""",652
8593,"""Ajax""",647
9931,"""FC Basel""",619
8456,"""Manchester City""",606
8455,"""Chelsea""",583
10260,"""Manchester United""",582
